In [36]:
import csv
import pandas as pd
data=pd.read_csv("train.csv")

In [37]:
from sklearn.feature_selection import SelectFromModel
import numpy as np

In [38]:
data = data.loc[:, data.any()]
from sklearn.preprocessing import StandardScaler
col=len(data.columns)
print(col)



42


In [39]:
ft=[str(i) for i in range(1,col+1) if str(i) in data.columns]
x_train=data[ft]
y_train=data["relevance"]
count=0
for val in y_train:
    if(val==1):
        count+=1
print(count)

8496


In [40]:
data2=pd.read_csv("val.csv")
x_val=data2[ft]
y_val=data2["relevance"]

In [41]:
scaler = StandardScaler()
print(scaler.fit(x_train))

StandardScaler()


In [42]:
print(scaler.mean_)

[1.42418604e-01 2.23062108e-01 2.07793004e-01 5.60512317e-01
 1.63602921e-01 1.48171453e-01 7.90023138e-02 6.34180912e-01
 2.16839888e-01 1.42748561e-01 5.54721706e-01 1.02126879e-01
 1.61305701e-01 8.26668388e-02 7.90536737e-02 5.68297176e-01
 2.81100621e-01 4.05640193e-01 2.26273026e-01 2.31574937e-01
 1.43390108e-01 3.99338033e-01 2.56693853e-01 6.30025826e-01
 2.57490088e-01 4.95699067e-01 1.43153849e-01 8.21807771e-02
 1.37527991e-01 5.52403049e-01 1.78322418e-01 1.64971546e-01
 5.69395677e-01 2.39389079e-05 1.42710382e-01 5.63985687e-01
 2.53889155e-01]


In [66]:
import lightgbm as lgb
gbm = lgb.LGBMClassifier(n_estimators=1000, min_split_gain=0.0,reg_lambda=0.8, silent=True,learning_rate=0.5,num_leaves=31,subsample_for_bin=200000,min_child_samples=20,min_child_weight=0.05,reg_alpha=0.5,penalty='l1')

In [67]:
sel_ = SelectFromModel(gbm)
sel_.fit(scaler.transform(x_train.fillna(0)), y_train)

SelectFromModel(estimator=LGBMClassifier(learning_rate=0.5,
                                         min_child_weight=0.05,
                                         n_estimators=1000, penalty='l1',
                                         reg_alpha=0.5, reg_lambda=0.8))

In [69]:
selected_feat = x_train.columns[(sel_.get_support())]

In [70]:
X_train_selected = sel_.transform(x_train.fillna(0))
X_val_selected = sel_.transform(x_val.fillna(0))

In [71]:
gbm.fit(X_train_selected, y_train, eval_set=[(X_val_selected, y_val)], early_stopping_rounds=5, verbose=False)

LGBMClassifier(learning_rate=0.5, min_child_weight=0.05, n_estimators=1000,
               penalty='l1', reg_alpha=0.5, reg_lambda=0.8)

In [72]:
test_pred = gbm.predict(X_val_selected)

In [73]:
from sklearn.metrics import f1_score
scor=f1_score(y_val, test_pred, average='macro')
print(scor)

0.48167798388028876
